# Finetuning Adagrad Notebook
------------------
This notebook contains the code used to train the models with the Adagrad optimizer. 

It functions primarily on Google Colab, but can be adapted to work on local hardware.

It trains only one type of model and searches through hyperparameters with a Grid Search algorithm.

For each model type, minimal adjustments need to be made before running the code again.

In [ ]:
# This script mounts Google Drive in a Google Colab environment.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Set the current working directory to a specific path in Google Drive.
%cd /content/drive/MyDrive/OptiML/repo/OptML-project

/content/drive/MyDrive/OptiML/repo/OptML-project


## Global Imports

In [ ]:
# Import necessary libraries for the project.
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets, transforms
from Functions.implementations import * # Import custom implementations, these contain the model definitions and training functions.

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Functions')

from implementations import *

## Load Data

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SimpleCNN().to(device)
train_loader,valid_loader, test_loader = get_data_loaders()

100%|██████████| 170M/170M [00:04<00:00, 41.9MB/s]


Train and evaluate a simple model to test if everything works.

# Adagrad :

In [ ]:
import os

#only for google Colab :

# Set a path in your Google Drive
csv_path = "/content/drive/MyDrive/OptML-project_ADAGRAD_DenseNet.csv"
save_path = "/content/drive/MyDrive/Results"
# For local machine, set the path to your desired location


epochs = 100
eval_interval = 10

# Create the CSV with headers if it doesn't exist
if not os.path.exists(csv_path):
    columns = ["learning_rate", "weight_decay", "lr_decay"] +  [f"epoch_{i}" for i in range(epochs//eval_interval, epochs + 1, eval_interval)] + ["Test"]
    pd.DataFrame(columns=columns).to_csv(csv_path, index=False)

set_seed(42) # Set a random seed for reproducibility

In [ ]:
from itertools import product

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_loader, valid_loader, test_loader = get_data_loaders(batch_size=128)

results_grid = {}

# Define the hyperparameters for the grid search
learning_rates = [5e-2, 1e-2, 5e-3]
weight_decays = [0.0, 1e-3, 1e-4]
lr_decays = [0.0, 1e-3]

# Iterate over all combinations of hyperparameters
for lr, wd, lr_decay in product(learning_rates, weight_decays, lr_decays):
    scores, model = train_and_return_evaluation_Adagrad(
        get_densenet121,  # Model type that needs to be trained, needs to be changed for different model types. Options are defined in the implementations.py file.
        lr=lr,
        lr_decay=lr_decay,
        weight_decay=wd,
        train_loader=train_loader,
        valid_loader=valid_loader,
        test_loader=test_loader,
        device=device,
        epochs=epochs,
        eval_interval=eval_interval)

    # Store the results in the grid
    results_grid[(lr, wd, lr_decay)] = scores

    # Extrac F1 scores for the CSV
    f1_scores = [f1 for (_, _, _, f1) in scores]
    row = [lr, wd, lr_decay] + f1_scores

    df_row = pd.DataFrame([row])
    df_row.to_csv(csv_path, mode='a', header=False, index=False)

    # Saving the model
    torch.save(model.state_dict(), save_path + f"/DenseNet_Adagrad_lr_{lr}_wd_{wd}_decay_{lr_decay}.pth")



🔧 Training with Adagrad: lr=0.05, lr_decay=0.0, weight_decay=0.001
Epoch 10 | Acc=0.7452 | Recall=0.7453 | F1=0.7457
Epoch 20 | Acc=0.8142 | Recall=0.8146 | F1=0.8136
Epoch 30 | Acc=0.8206 | Recall=0.8218 | F1=0.8225
Epoch 40 | Acc=0.8498 | Recall=0.8503 | F1=0.8508
Epoch 50 | Acc=0.8282 | Recall=0.8291 | F1=0.8315
Epoch 60 | Acc=0.8646 | Recall=0.8645 | F1=0.8646
Epoch 70 | Acc=0.8626 | Recall=0.8628 | F1=0.8622
Epoch 80 | Acc=0.8670 | Recall=0.8674 | F1=0.8676
Epoch 90 | Acc=0.8546 | Recall=0.8552 | F1=0.8560
Epoch 100 | Acc=0.8664 | Recall=0.8662 | F1=0.8653

Test Set Evaluation: Acc=0.8634 | Recall=0.8634 | F1=0.8619

🔧 Training with Adagrad: lr=0.01, lr_decay=0.0, weight_decay=0.001
Epoch 10 | Acc=0.8032 | Recall=0.8016 | F1=0.7983
Epoch 20 | Acc=0.8808 | Recall=0.8810 | F1=0.8805
Epoch 30 | Acc=0.8908 | Recall=0.8903 | F1=0.8910
Epoch 40 | Acc=0.9074 | Recall=0.9073 | F1=0.9072
Epoch 50 | Acc=0.8974 | Recall=0.8961 | F1=0.8969
Epoch 60 | Acc=0.8878 | Recall=0.8884 | F1=0.8886
Ep